In [240]:
%load_ext autoreload
%autoreload 2

from tweedejaars_project import *
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [241]:
features = [
    'import_capacity',
    'min_price_published',
    'mid_price_published',
    'max_price_published',
    # 'min_ptu_price_known',
    # 'max_ptu_price_known',
    # 'settlement_price_bestguess',
    'time_since_last_two_sided',
    'two_sided_daily_count',
    'PTU',
    'naive_strategy_action',
    'forecast_wind',
    'forecast_solar',
    'forecast_demand',
    'ptu_id',
    'fix_two_sided_ptu_realtime'
]

# already used
target = 'target_two_sided_ptu'

In [242]:
df = load_df()
splits = get_splits(df, features)

In [244]:
# Hyperparameters
sequence_length = 50 # Dit is het PTU aantal dus ptu_id's niet row aantal
batch_size = 32
input_size = len(features)
hidden_size = 64
num_layers = 2
output_size = 1
num_epochs = 10
learning_rate = 0.001


# Split the data in vars
train_data = splits['train']
test_data = splits['test']


In [245]:
# def create_sequences(seq_length=50, ptu_length=15):
#     sequence_indices = []
#     ptu_ids_total = train_data['in']['ptu_id']
#     ptu_ids = ptu_ids_total.unique()

#     for ptu_id in ptu_ids:
#         last_idx_start_ptu = np.searchsorted(ptu_ids_total, ptu_id, side='left')

#         first_idx = last_idx_start_ptu - (seq_length * ptu_length)
        
#         if first_idx < 0:
#             first_idx = 0

#         # DIT IS EEN HACK WANT IK NEGEER NU DE EERSTE OM EEN GOEDE LENGTE TE HEBBEN VOOR ELKE INPUT
#         # TODO: KAN WORDEN OPGELOST MET PADDING?
#         for row_idx in range(ptu_length):
#             last_idx = last_idx_start_ptu + row_idx
#             sequence_indices.append((first_idx, last_idx))

#     # niet meer nodig die id
#     return sequence_indices

# sequence_indices = create_sequences()
# sequence_indices

In [246]:
# # TEST CODE 
# def create_sequences(data_in, seq_length, ptu_length):
#     sequence_indices = []
#     ptu_ids_total = data_in['ptu_id']
#     ptu_ids = ptu_ids_total.unique()
    
#     for ptu_id in ptu_ids:
#         last_idx_start_ptu = np.searchsorted(ptu_ids_total, ptu_id, side='left')

#         for row_idx in range(ptu_length):
#             last_idx = last_idx_start_ptu + row_idx
#             first_idx = last_idx - (seq_length * ptu_length) + 1
            
#             if first_idx < 0:
#                 first_idx = 0
                
#             sequence_indices.append((first_idx, last_idx))

#     return sequence_indices

# # Function to pad sequences
# def pad_sequences(sequence, target, seq_length, ptu_length, num_features, nan_val1, nan_val2):
#     max_rows = seq_length * ptu_length
#     num_rows = sequence.shape[0]
#     pad_rows = max(0, max_rows - num_rows)

#     if pad_rows > 0:
#         padding_sequence = np.full((pad_rows, num_features), fill_value=np.nan, dtype=np.float32)
#         sequence = np.vstack((padding_sequence, sequence))
#         padding_target = np.full((pad_rows,), fill_value=np.nan, dtype=np.float32)
#         target = np.concatenate((padding_target, target))
    
#     return sequence, target

# # Function to test sequence creation and padding
# def test_sequences(train_data, seq_length, ptu_length, nan_val1, nan_val2):
#     sequence_indices = create_sequences(train_data['in'], train_data['out'], seq_length, ptu_length)
#     for idx, (start_idx, end_idx) in enumerate(sequence_indices):
#         sequence = train_data['in'].iloc[start_idx:end_idx + 1].drop('ptu_id', axis=1).values
#         target = train_data['out'].iloc[start_idx:end_idx + 1].values
#         sequence, target = pad_sequences(sequence, target, seq_length, ptu_length, sequence.shape[1], nan_val1, nan_val2)
#         print(f"Sequence {idx}: {sequence.shape}, Target: {target.shape}, idx's [{start_idx, end_idx}]")

# # Hyperparameters

# # test_sequences(train_data, seq_length, ptu_length, nan_val1, nan_val2)


In [247]:
    # def create_sequences(self):
    #     sequence_indices = []
    #     ptu_ids_total = self.data_in['ptu_id']
    #     ptu_ids = ptu_ids_total.unique()

    #     for ptu_id in ptu_ids:
    #         last_idx_start_ptu = np.searchsorted(ptu_ids_total, ptu_id, side='left')

    #         first_idx = last_idx_start_ptu - (self.seq_length * self.ptu_length)
            
    #         if first_idx < 0:
    #             first_idx = 0

    #         # DIT IS EEN HACK WANT IK NEGEER NU DE EERSTE OM EEN GOEDE LENGTE TE HEBBEN VOOR ELKE INPUT
    #         # TODO: KAN WORDEN OPGELOST MET PADDING?
    #         # if first_idx > 0:
    #         for row_idx in range(self.ptu_length):
    #             last_idx = last_idx_start_ptu + row_idx
    #             sequence_indices.append((first_idx, last_idx))

    #     # niet meer nodig die id
    #     self.data_in = self.data_in.drop('ptu_id', axis=1)
    #     return sequence_indices


In [295]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_length, ptu_length=15, nan_val1 = 100000, nan_val2 = -100000):
        self.data_in = data['in'].astype(np.float32)
        self.data_out = data['out'].astype(np.float32)
        self.seq_length = seq_length
        self.ptu_length = ptu_length
        self.sequence_indices = self.create_sequences()

        # DIT IS EEN HACK DUS WSS VERANDEREN
        self.data_in = self.data_in.fillna({'min_price_published':nan_val1, 'max_price_published':nan_val2}).astype(np.float32)

    def create_sequences(self):
        sequence_indices = []
        ptu_ids_total = self.data_in['ptu_id']
        ptu_ids = ptu_ids_total.unique()
        
        for ptu_id in ptu_ids:
            last_idx_start_ptu = np.searchsorted(ptu_ids_total, ptu_id, side='left')

            for row_idx in range(self.ptu_length):
                last_idx = last_idx_start_ptu + row_idx
                first_idx = last_idx - (self.seq_length * self.ptu_length) + 1
                
                if first_idx < 0:
                    first_idx = 0
                    
                sequence_indices.append((first_idx, last_idx))

        return sequence_indices
    
    # sequence, target, seq_length, ptu_length, num_features, nan_val1, nan_val2
    # def pad_sequences(self, sequence, target):
    #     max_rows = self.seq_length * self.ptu_length
    #     num_rows = sequence.shape[0]
    #     num_features = sequence.shape[1]
    #     pad_rows = max(0, max_rows - num_rows)

    #     # FF opletten met die np.nan idk hoe goed dit gaat werken
    #     if pad_rows > 0:
    #         padding_sequence = np.full((pad_rows, num_features), fill_value=np.nan, dtype=np.float32)
    #         sequence = np.vstack((padding_sequence, sequence))
    #         padding_target = np.full((pad_rows,), fill_value=np.nan, dtype=np.float32)
    #         target = np.concatenate((padding_target, target))

        
    #     return sequence, target

    def __len__(self):
        return len(self.sequence_indices)
    
    def __getitem__(self, idx):
        start_idx, end_idx = self.sequence_indices[idx]
        sequence = self.data_in.iloc[start_idx:end_idx + 1].drop('ptu_id', axis=1).values
        target = self.data_out.iloc[start_idx:end_idx + 1].values
            
        return torch.tensor(sequence, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)


In [296]:
def prepare_data_loader(data, sequence_length, batch_size):
    dataset = TimeSeriesDataset(data, sequence_length)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)


In [310]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, lengths):
        # Calculate the actual lengths of each sequence in the batch
        lengths = torch.count_nonzero(~torch.isnan(x), dim=1).cpu()
        
        # Sort sequences by length
        sorted_lengths, sorted_indices = lengths.sort(descending=True)
        x_sorted = x[sorted_indices]
        
        # Pack the padded batch of sequences
        packed_input = nn.utils.rnn.pack_padded_sequence(x_sorted, sorted_lengths, batch_first=True, enforce_sorted=False)
        
        # Forward propagate RNN
        packed_output, _ = self.rnn(packed_input)
        
        # Unpack the output
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        
        # Decode the hidden state of the last time step
        out = self.fc(output)
        out = torch.sigmoid(out[:, -1, :])
        
        return out


In [347]:
import torch.nn.functional as F
source = torch.rand((42,3))
print(source)
# now we expand to size (7, 11) by appending a row of 0s at pos 0 and pos 6, 
# and a column of 0s at pos 10

max_length = 50
result = F.pad(input=source, pad=(0, 0, max(0, max_length - source.shape[0]), 0), mode='constant', value=np.nan)
print(result.shape)
filtered_tensor = result[~torch.all(result.isnan(),dim=1)]
print(filtered_tensor.shape)

tensor([[0.9658, 0.1616, 0.0534],
        [0.6900, 0.0018, 0.8960],
        [0.5663, 0.5930, 0.5741],
        [0.6836, 0.2389, 0.4567],
        [0.6753, 0.9381, 0.0526],
        [0.3978, 0.5444, 0.9855],
        [0.3204, 0.9567, 0.2313],
        [0.7106, 0.2493, 0.7830],
        [0.3562, 0.2846, 0.2962],
        [0.8869, 0.8583, 0.4654],
        [0.1842, 0.6890, 0.4974],
        [0.7305, 0.3030, 0.4937],
        [0.8837, 0.8813, 0.5206],
        [0.7902, 0.0288, 0.3473],
        [0.9116, 0.9858, 0.7223],
        [0.7183, 0.4518, 0.8939],
        [0.6305, 0.1593, 0.4431],
        [0.8508, 0.2299, 0.3121],
        [0.5648, 0.2167, 0.3083],
        [0.3356, 0.4826, 0.2622],
        [0.5975, 0.3899, 0.3307],
        [0.3482, 0.7138, 0.1033],
        [0.0601, 0.1090, 0.9275],
        [0.7493, 0.8437, 0.2948],
        [0.5256, 0.3413, 0.8827],
        [0.8071, 0.3404, 0.3854],
        [0.9666, 0.1025, 0.0826],
        [0.0135, 0.3791, 0.5936],
        [0.6137, 0.0087, 0.6421],
        [0.851

In [365]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence

# Generate sequences of varying lengths
sequences = [torch.rand((42, 5)), torch.rand((30, 5)), torch.rand((35, 5))]
desired_length = 50

# Manually pad sequences to the desired length
def pad_to_length(sequences, length, padding_value=np.nan):
    padded_sequences = []
    for seq in sequences:
        if seq.shape[0] < length:
            pad_length = length - seq.shape[0]
            padded_seq = F.pad(seq, (0, 0, pad_length, 0), mode='constant', value=padding_value)
        else:
            padded_seq = seq[:length]
        padded_sequences.append(padded_seq)
    x = pad_sequence(padded_sequences, batch_first=True, padding_value=padding_value)
    print(x.squeeze(0).shape, 'hallo')
    return x

# Pad sequences to the desired length
padded_sequences_tensor = pad_to_length(sequences, desired_length)
print("Padded sequences shape:", padded_sequences_tensor.shape)

# Define a simple RNN
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x, lengths):
        # Pack the padded sequence
        packed_input = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.rnn(packed_input)
        # Unpack the sequence
        output, _ = pad_packed_sequence(packed_output, batch_first=True, total_length=desired_length)
        return output

# Get lengths of original sequences (before padding)
sequence_lengths = [min(seq.shape[0], desired_length) for seq in sequences]

# Instantiate the RNN with appropriate dimensions
input_size = padded_sequences_tensor.shape[2]  # Number of features (5 in this case)
hidden_size = 20  # Number of features in the hidden state
num_layers = 1  # Number of recurrent layers
rnn = SimpleRNN(input_size, hidden_size, num_layers)

# Pass the padded sequences and their lengths to the RNN
output = rnn(padded_sequences_tensor, sequence_lengths)
print("RNN output shape:", output.shape)



torch.Size([3, 50, 5]) hallo
Padded sequences shape: torch.Size([3, 50, 5])
RNN output shape: torch.Size([3, 50, 20])


In [311]:
train_loader = prepare_data_loader(train_data, sequence_length, batch_size)

In [312]:
model = RNNModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [313]:
for epoch in range(num_epochs):
    for inputs, masks, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs, masks)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


IndexError: index_select(): Index is supposed to be a vector